## 数据集下载

In [ ]:
from datasets import load_dataset


ds_reason = load_dataset("Morefreedai/medical-dpo-v1",cache_dir='./data/zh')
ds_no_reason = load_dataset("ShelterW/Medical-R1-dpo-pairs",cache_dir = './data/en')

## 数据集处理

In [ ]:
import json
import random

# 设置随机种子确保可复现
random.seed(42)

# 准备训练集和测试集
train_data = []
test_data = []

# 处理英文数据集 (ds_en) 的训练集
english_data = list(ds_en['train'])  # 转换为list便于采样
random.shuffle(english_data)

# 抽取65条英文测试数据
for i, item in enumerate(english_data[:65]):
    test_data.append({
        'question': item['chosen'][0]['content'],
        'cot': None,
        'answer': item['chosen'][1]['content'],
        'type': 'en',
        'dialog_id': i
    })

# 剩余英文数据加入训练集
for item in english_data[65:]:
    train_data.append({
        'prompt': item['chosen'][0]['content'],
        'chosen': item['chosen'][1]['content'],
        'rejected': item['rejected'][1]['content'],
    })

# 处理中文数据集 (ds_zh) 的data部分
chinese_data = list(ds_zh['data'])
random.shuffle(chinese_data)

# 抽取65条中文测试数据
for j, item in enumerate(chinese_data[:65]):
    test_data.append({
        'question': item['prompt'],
        'cot': None,
        'answer': item['chosen'],
        'type': 'zh',
        'dialog_id': j + 65  # 英文用了0-64，中文从65开始
    })

# 剩余中文数据加入训练集
for item in chinese_data[65:]:
    train_data.append({
        'prompt': item['prompt'],
        'chosen': item['chosen'],
        'rejected': item['rejected'],
    })

# 打乱训练集顺序
random.shuffle(train_data)

# 保存训练集
with open('train_preference_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

# 保存测试集
with open('test_qa_data.json', 'w', encoding='utf-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)

print(f"处理完成：")
print(f"- 训练集：{len(train_data)} 条（英文{len(english_data)-65}条 + 中文{len(chinese_data)-65}条）")
print(f"- 测试集：{len(test_data)} 条（英文65条 + 中文65条）")

## llama factory

### 训练集

In [ ]:
import json

# 1. 加载原始训练集
with open('train_preference_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

# 2. 转换为 ShareGPT 格式
sharegpt_data = []
for item in train_data:
    # 构建对话部分（严格匹配示例格式）
    conversations = [
        {
            "from": "human",
            "value": item["prompt"]  # 假设原字段是 prompt
        },
        {
            "from": "gpt",
            "value": item["chosen"]
        }
    ]
    
    # 构建 chosen/rejected（严格嵌套格式）
    chosen = {
        "from": "gpt",
        "value": item["chosen"]
    }
    rejected = {
        "from": "gpt",
        "value": item["rejected"]
    }
    
    sharegpt_data.append({
        "conversations": conversations,
        "chosen": chosen,
        "rejected": rejected
    })

# 3. 保存结果
with open('train_sharegpt.json', 'w', encoding='utf-8') as f:
    json.dump(sharegpt_data, f, ensure_ascii=False, indent=2)

print(f"转换完成，共处理 {len(sharegpt_data)} 条训练数据")

### 测试集

In [ ]:
import json

def convert_test_to_alpaca(test_data):
    alpaca_data = []
    for item in test_data:
        # 提取instruction和output（假设question作为instruction，answer作为output）
        alpaca_item = {
            "instruction": item["question"]+" /no_think",
            "input": "",  # 留空，如示例所示
            "output": "<think>\n\n</think>\n"+item["answer"]
        }
        alpaca_data.append(alpaca_item)
    return alpaca_data

# 加载原始测试集
with open('test_qa_data.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

# 转换测试集
alpaca_test = convert_test_to_alpaca(test_data)

# 保存Alpaca格式测试集
with open('test_alpaca.json', 'w', encoding='utf-8') as f:
    json.dump(alpaca_test, f, ensure_ascii=False, indent=2)

print(f"测试集转换完成，共处理 {len(alpaca_test)} 条数据")

### 把数据移动到llama factory的adta目录下

mv train_sharegpt.json LLaMA-Factory/data

mv test_alpaca.json LLaMA-Factory/data

### 在dataset_info.json中添加
